# Overview

In this programming assignment, you will be applying the knowledge that you have learned in the lectures to build a simple indexing and retrieval system for a search engine. This assignment should be done in teams of two or individually. Assignments are graded the same for one and two person teams. 

More specifically, it involves the following tasks:
1. [Build an uncompressed index (45%)](#Building-an-uncompressed-index-and-retrieval-(45%)) over a corpus of webpages from the stanford.edu domain, and implement retrieval for Boolean conjunctive queries.
2. [Build a compressed index (30%)](#Building-a-compressed-index-(30%)) over the same corpus using variable length encoding and implement Boolean conjunctive queries.
3. [Write up a report (25%)](#Report-(25%)) describing your program and answer a set of questions.
4. [For extra credit (10%)](#Additional-compression-methods-(10%---Extra-credit)) you are encouraged to implement other compression algorithms (e.g., gamma-encoding).


## Submission instructions

1\. The assignment is due before class at 4:00 pm on the due date (23rd April 2019)

2\. The notebook will automatically generate **python files** in submission folder. You'll have to upload them to the PA1-code assignment on gradescope. Note that you need to upload all the individual files in the submission folder without zipping it.    

3\. While solving the assignment, do **NOT** change class and method names, autograder tests will fail otherwise. 

4\. You'll also have to upload a **PDF version** of the notebook (which would be primarily used to grade your report section of the notebook) to PA1-PDF assignment on gradescope. Note that directly converting the PDF truncates code cells. To get a usable PDF version, first click on File > Print Preview, which will open in a new tab, then print to PDF using your browser's print functionality. 

5\. After uploading the PDF make sure you **tag all the relevant pages to each question**. We will penalize for mistagged submissions. 

6\. If you are solving the assignment in a team of two, add the other student as a group member after submitting the assignment. Do **NOT** submit the same assignment twice. 

## Setup

In [ ]:
#Load the tee magic which saves a copy of the cell when executed
%reload_ext autograding_magics

The `submission` folder will contain all the files to be submitted.

In [ ]:
import os
try: 
    os.mkdir('submission')
except FileExistsError:
    pass

In [ ]:
%%tee submission/imports.py

# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib

# Corpus

The corpus you will be working with for this assignment contains webpages from the stanford.edu domain. The data for this assignment is available as a .zip file at: http://web.stanford.edu/class/cs276/pa/pa1-data.zip. The following code puts the data folder under the current directory. The total size of the corpus is about 170MB.

In [ ]:
import urllib.request
import zipfile

data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
data_dir = 'pa1-data'
urllib.request.urlretrieve(data_url, data_dir+'.zip')
zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()

The index will be stored in `output_dir` and `tmp` will contain some temporary files for toy indices.

In [ ]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

There are 10 sub-directories (named 0-9) under the data directory.

In [ ]:
sorted(os.listdir('pa1-data'))

Each file under the sub-directory is the content of an individual webpage. You should assume that the individual file names are unique within each sub-directory, but not necessarily the case across sub-directories (*i.e.,* the full path of the files are unique).

In [ ]:
sorted(os.listdir('pa1-data/0'))[:10]

All the HTML info has been stripped off from those webpages, so they only contain space-delimited words. You should not re-tokenize the words. Each consecutive span of non-space characters consists of a word token in the corpus.

In [ ]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

You will also find a small corpus included with the assignment in the folder `toy-data`. We will use this toy dataset to test our code before running on the complete dataset.

In [ ]:
toy_dir = 'toy-data'

# Building an uncompressed index and retrieval (45%)

The first part of this assignment is to build an inverted index of this corpus, and implement Boolean conjunctive queries. **In particular, you need to implement the blocked sort-based indexing (BSBI) algorithm described in [Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf) of the textbook.** While we quote some fundamentals from the textbook, we strongly suggest that you read Section 4.2 in more detail to fully understand the BSBI algorithm.

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

In this assignment, you will be implementing methods for large collections that require the use of secondary storage (*i.e.,* disk).

## IdMap

Again quoting from Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

Let us first build a helper class `IdMap`, which maps strings to numeric ids (and vice-versa). We will need this to bijectively map term to termID and doc to docID.

Fill in the functions `_get_str` and `_get_id` in the following code. The only interface to this class is provided by `__getitem__` which gets the correct mapping depending on the type of the key. (See [these docs](https://docs.python.org/3.7/reference/datamodel.html#special-method-names) if you are unfamiliar with special functions like `__getitem__` ). Further, to simplify code later on, we'll also add some logic to add a string if it doesn't already exist in the map.   

We will use a dictionary for efficient access from strings to numeric ids, and we will use a list for efficient access (and storage) from numeric ids to strings.

You might also find this [short tutorial](https://www.omkarpathak.in/2018/04/11/python-getitem-and-setitem/) useful to get started with special (a.k.a magic) methods


In [ ]:
%%tee submission/id_map.py
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code

        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code

        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

Make sure it passes the following simple test cases

In [ ]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

Going forward we will expect you to write your own test cases to make sure parts of your program are working as expected.

## Encoding Postings List as bytearrays

In order to write and read lists of postings (docIDs) efficiently from the disk, we store them as bytearrays. We've provided an implementation of `UncompressedPostings` class which contains static encode and decode functions. In the next task you'll be required to implement compressed versions with the same inferface. 

For reference:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [ ]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

To illustrate how it works, run the following cell

In [ ]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

## Inverted Index on Disk

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

In this section we provide a base class `InvertedIndex` which would be subsequently subclassed into `InvertedIndexWriter`, `InvertedIndexIterator` and `InvertedIndexMapper`. While typically `cPickle` is used for serialization in Python, it does not support partial reads and writes, which is why we are rolling out our own custom solution.

In [ ]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their 
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion. 
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """

        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = {}
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onwards

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

Since we are interacting with a file on disk, we provide `__enter__` and `__exit__` functions, which allow using the `with` statement just like you would with file IO in python (Refer to the [official documentation for context managers](https://docs.python.org/3/library/contextlib.html)).

Here is an example of how to use the `InvertedIndexWriter` context manager:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

If you are unfamiliar with context managers in python, we would recommend checking out these tutorials ([1](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/), [2](http://arnavk.com/posts/python-context-managers/)), although it isn't necessary to be able to understand all of it.

## Indexing

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

You should treat each sub-directory as a block, and only load one block in memory at a time when you build your index (note: you will be penalized if you construct the index by loading blocks of larger than one directory in memory at once). Note that we are abstracting away from the operating systems meaning of blocks. You can assume that each block is small enough to be stored in memory.

In this section, we introduce the class `BSBIIndex` which we will progressively build. The `index` function provides the skeleton for BSBI and your job is to implement `parse_block`, `invert_write` and `merge` functions in subsequent sections.

In [ ]:

# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

### Parsing

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

You should treat each sub-directory as a block; `parse_block` expects the path to the sub-directory which is the block. You should assume that the individual file names are unique within each sub-directory, but not necessarily the case across sub- directories (i.e., the full path of the files are unique).

_NB - We are inheriting `BSBIIndex` from `BSBIIndex`, which is just a simple way of adding a new method to an existing class. Please do not be confused, it is simply used as a pedagogical tool to split a class definition within a jupyter notebook and there is no other magic happening here._

In [ ]:
%%tee submission/parse_block.py
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code

        ### End your code

See if the function works as expected on the toy data.

In [ ]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

In [ ]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')

Does the `parse_block` method work as expected on a block of the toy data? Write some tests to make sure that is indeed the case (for example, you should make sure that a given word gets the same id each time it appears).

In [ ]:
### Begin your code

### End your code

### Inversion

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

In this section we add the function `invert_write` which does exactly this. 

However, we first need to implement the class `InvertedIndexWriter`. This class provides an append function, just like a list, however the postings_list isn't stored in memory but is directly written to disk.

In [ ]:
%%tee submission/inverted_index_writer.py
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code

        ### End your code

Even though we haven't written the classes which read the index, we can still test the implementation as follows. Make sure you pass the asserts before moving forward

In [ ]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

Now we implement `invert_write`, which takes in the td_pairs created with parse_block. We use `InvertedIndexWriter` to write them to disk.

In [ ]:
%%tee submission/invert_write.py
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code

        ### End your code

To test this on the toy data, we can read a block and see what the inverted index contains. Write some tests like you saw for the `InvertedIndexWriter`.

In [ ]:
### Begin your code

### End your code

### Merging

> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

The iterator paradigm in Python lends itself naturally to our need of maintaining a small read buffer. We can iterate through the file while reading just one postings list at a time from the disk. We subclass `InvertedIndex` into `InvertedIndexIterator` which does the iteration bit. 

In [ ]:
%%tee submission/inverted_index_iterator.py
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code

        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.
        """
        ### Begin your code

        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

Let's test this by first creating an index with the `InvertedIndexWriter` and then iterating through it. Write a few small test cases to see if it works. At the very least, you should write a test which manually constructs a small inverted index, writes them to disk with an `InvertedIndexWriter`, then uses an `InvertedIndexIterator` to iterate over the inverted index.

In [ ]:
### Begin your code

### End your code

> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

We'll use the `InvertedIndexIterator` to do the reading part and `InvertedIndexWriter` to write the merged postings list. 

Note that we've been opening each inverted index using `with` statement, but now we need to do that programmatically. You can see that we've used `contextlib` ([documentation](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack)) for that in the provided `index` function of `BSBIIndex` base class. Your task is to **write the logic** for merging *opened* `InvertedIndexIterator` objects and writing one postings list at a time into into a single `InvertedIndexWriter` object. 

Since we know that the postings are sorted, we can appropriately merge them in sorted order in linear time. In fact `heapq` ([documentation](https://docs.python.org/3.0/library/heapq.html)) is a standard python module that provides an implementation of the heap queue algorithm. In particular it contains a `heapq.merge` utility function which merges multiple sorted inputs into a single sorted output and returns an iterator over the sorted values. Not only can this be handy with merging postings lists, but also with merging the inverted indices. 

To get you started on using the `heapq.merge` function, we've provided a sample usage of the function. The two lists contain animals/birds sorted by their average life span. We want to merge the two lists.

In [ ]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

Notice the use of `*` to unpack `lifespan_lists` as arguments and `lambda` function to reprsent the key to perform the soring with. If you are unfamiliar you can check documentation ([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions)) or tutorials ([unpacking lists](https://www.geeksforgeeks.org/packing-and-unpacking-arguments-in-python/), [lambda_expressions](https://www.afternerd.com/blog/python-lambdas/))

In [ ]:
%%tee submission/merge.py

import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
        ### Begin your code

        ### End your code

First make sure it works without errors on toy data

In [ ]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', )
BSBI_instance.index()

Now lets index the full corpus

In [ ]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.index()

If you're having errors with just the merging part of it, you can use the following code to debug it. 

In [ ]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)

## Boolean conjunctive retrieval

We'll be implementing a function `retrieve` to BSBIIndex, which given a query string consisting of space separated tokens, returns a list of documents that contain each of the tokens in the query. However, we do not want to be iterating through the index nor loading the entire index to find the relevant terms. 

First we'll implement `InvertedIndexMapper` which subclasses `InvertedIndex` to add functionality for retrieving postings corresponding to a particular term by seeking to that location in the file.

In [ ]:
%%tee submission/inverted_index_mapper.py
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code

        ### End your code

Write a few tests to check your `_get_postings_list` implementation.

In [ ]:
### Begin your code

### End your code

Now that we can obtain postings lists corresponding to the query terms, we want to intersect them. However, à la merging, we can use the sorted-ness of these lists to intersect them very efficiently. Here we'll implement `sorted_intersect` which takes two sorted lists and return a sorted intersection of the elements.  

In [ ]:
%%tee submission/sorted_intersect.py
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code

    ### End your code

Write a few simple test cases to make sure it works correctly

In [ ]:
### Begin your code

### End your code

Now we're ready to write the `retrieve` function using `InvertedIndexMapper` and `sorted_intersect`

In [ ]:
%%tee submission/retrieve.py
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query
        
        Parameters
        ----------
        query: str
            Space separated list of query tokens
            
        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens. 
            Should be empty if no documents are found.
        
        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()

        ### Begin your code

        ### End your code

Let us test if our index works on the real corpus with a simple query

In [ ]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.retrieve('boolean retrieval')

You can also test if one of these pages truly contains the query terms by reading a file as follows

In [ ]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

Let's test on the dev queries to see if it works alright. 

In [ ]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

If an assert is failing you can compare the difference in outputs as follows

In [ ]:
set(my_results) - set(reference_results)

In [ ]:
set(reference_results) - set(my_results)

Make sure you write your own queries test for all kinds of corner cases like terms which have never occurred in the corpus, or terms that are very frequent and might slow down the merge. 

### Grading breakdown
To ensure your index is built correctly, you will be tested on 20 Boolean conjunctive queries of one or multiple terms. For those queries, you will get 1.5% of the final grade for each query you answer correctly, for a total of 30% of your grade. 8 of those queries will be the same as the provided dev queries. 

Note that we'll be running your submission in an underpowered VM (to simulate pairity with realistic conditions where you would have larger block sizes and more memory) with around 750 MB of available memory. If your program uses more than that, it will be killed by the VM. You should make sure well ahead of time that your submission works and passes the visible tests and queries upon submitting to the gradescope autograder.

Further we'll also run a few tests to check if the intermediate and final index files generated match the reference implementation and also a few unit tests to check correctness of each of the functions asked to implement. Note that due to the ordering constraints there is exactly one correct output expected. You will earn the remaining 15% of the points based on that. The breakdown is as follows

1. `IdMap` - 2%
2. `parse_block` - 2%
3. `InvertedIndexWriter` - 1%
4. `invert_write` - 2%
5. `InvertedIndexIterator` - 1%
6. `merge` - 2%
7. `InvertedIndexMapper` - 1%
8. `sorted_intersect` - 2%
9. `retrieve` - 2%

We will also collect timing statistics on how long the indexing takes, the size of your generated index, and the amount of memory used when building the index. 
Finally we'll inspect the code, generated indices and penalize by
1. 5% if your `retrieve` function loads the whole index into memory rather than simply loading the postings lists of just the query terms.
2. 5% if your `merge` function loads the complete intermediate indices or keeps the whole merged index in memory.
3. 4% if you do not order the query terms by postings list length in `retrieve` function.
4. 4% if you use the built in set operations (which do not explicitly use the sorteness of postings lists) while merging postings list in `merge` function or intersecting sorted lists in `sorted_intersect` function. 
5. 5% if the timing statistics of your retrieval algorithm (`retrieve` function) are way out of the norm (which might happen if you iterate through your index instead of seeking just the query terms).
6. 5% if the memory sizes of generated indices are way out of the norm 

The maximum combined grades for this section is 45%.

# Building a compressed index (30%)

In this section, you should build a compressed index using gap encoding with variable byte encoding for each gap.

Here are some things about python you'll find useful. 

In [ ]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

Fill in the following `CompressedPostings` class which we shall use as a drop-in replacement for `UncompressedPostings`. You should refer to [Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf) of the textbook for understanding how gap encoding with variable byte encoding for each gap is done. 

In [ ]:
%%tee submission/compressed_postings.py
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code

        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code

        ### End your code


First, write some test cases for any helper methods that you might have implemented

In [ ]:
### Begin your code

### End your code

We've provided a simple helper function for you to test whether an encoded postings list is decode correctly.

In [ ]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

Write a few test cases to make sure the postings list compression and decompression is being done correctly

In [ ]:
### Begin your code

### End your code

Now let us create a new output directory and use the `CompressedPostings` to create a compressed index

In [ ]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [ ]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.index()

In [ ]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

As before, let us now test on the dev queries to see if it works alright

In [ ]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert my_results == reference_results, "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

## Grading breakdown

Similar to uncompressed indexing, you will be tested on 20 conjunctive queries (1.5% for each query you
answer correctly, for a total of 30%). 

You will be penalized by:
1. 15% (of the total grade for the assignment) if your compression algorithm does not achieve a compressed index of size less than 20MB on disk (just the .index file without the .dict file).
2. 5% (of the total grade for the assignment) if your query response time with the compressed index is way out of norm.

The combined total grade of this section is 30%.   
*Note that if your program does not implement the variable length encoding compression algorithm, you will receive no credit for this task.*

## Code submission

Now you are ready to submit the code part of your assignment. Refer to [submission instructions section](#Submission-instructions). 

# Report (25%)

## Running time analysis (10%)

In this section your task is to write queries to probe running time characteristics of `retrieve` function (without compresssion) and use it to illustrate key steps and optimizations of the retrieval algorithm. 

Each of the following subsections have two deliverables, 
1. One or more queries 
2. Reasoning in the form of a written answer. Please limit your answer to 3 sentences.  

Each subsection is worth 2% of the assignment. 

**Written answers should be blockquotes**, so that we don't miss them while grading. You can edit a markdown cell by double clicking on it. See the last cell of the following sub-section for an example.

To get you started, we have provided with an example of timing a query (Note that the following cells assume you have an indexed BSBIIndex object)

You can read more about `%timeit` in the [documentation](https://docs.python.org/3/library/timeit.html)

In [ ]:
%timeit BSBI_instance.retrieve('boolean retrieval')

### `q1` - Slower query (2%)

How can you replace a term in the query so that it runs slower? Time the query and see if it behaves as per your hypothesis. We will refer to this query as `q1`


In [ ]:
### Begin your code

### End your code

**What is your reasoning behind the change? Why did the retrieve function slow down?**
> *Your answer here*

### `q2` - Twice as slow query (2%)

Can you make the retrieval twice as slow? Construct a query that runs twice as slow as your previous query (`q1`). We will refer to this query as `q2`

In [ ]:
### Begin your code

### End your code

**What is the general principle behind your construction of the query that made it run twice as slowly?**

> *Your answer here*

### `q3` - Confounding factors (2%)

The query that ran twice as slow (`q2`) might differ from its previous query (`q1`) in many ways. Thus the slowdown could have been caused due to any of those differences. 

Write one or more queries to show that that the above stated general principle is the sole cause for the slowdown and none of the other differences. 

In [ ]:
### Begin your code

### End your code

**What factors could you eliminate as a reason for slowdown? Describe how the queries eliminated those factors.** 

> *Your answer here*

Were there are any factors that you couldn't eliminate? Can you change `q2` so that it is still twice as slow as `q1` but no longer has confounding factors? If not, do you need to rethink your general principle for slowdown in `q2`?

### `q4` - Adding new terms (2%)

Construct a new query by adding a term to `q2` that makes it go faster than `q2` (though it'll likely be much slower than `q1`). We'll refer to it as `q4`

In [ ]:
### Begin your code

### End your code

**How did you do it? What is the principle behind the increase in speed?** 

> *Your answer here*

### `q5` - Effect of shuffling (2%)

Shuffle the terms in `q4` and time it

In [ ]:
### Begin your code

### End your code

**Did you notice a *significant* difference between timing for `q4` and `q5`? Why would you not expect the timing to change too much?** 

> *Your answer here*

Note that these timings are not perfect but you should be able to get significant differences where you would expect them without much effort

## Index size (6%) 

In this section we will look at the size of index on disk. Each subsection is worth 2% of the assignment. We will be grading both the analysis code and the written answer.

### Uncompressed index (2%) 

Let us look at the size of the merged index (`.index`) file 

In [ ]:
print("Size of index", os.path.getsize("output_dir/BSBI.index"), 'bytes')

The code above queries the file system to compute the size of the index. Show that you can compute the same answer by looking at the `postings_dict` object directly. In particular, write one line of Python code to compute the size of the index file (in bytes) using the postings dict.

**Hint:** You should use the word size (in bytes) and size information stored in the `postings_dict`.

In [ ]:
with open('output_dir/BSBI.dict', 'rb') as f:
    postings_dict, terms = pkl.load(f)
    
### Begin your code

### End your code

**How did you compute the expected size based on the information stored in the postings dictionary? What is the size of each integer representing a posting?**

> *Your answer here*

### Reducing size of uncompressed index (2%)

One way of reducing size of uncompressed index is to simply use fewer bits for each posting. What is the smallest number of bits that you would need to represent postings (i.e. docIds) for this document collection? In the following cell collect statistics that help you determine the answer

In [ ]:
### Begin your code

### End your code

**If each posting was represented by the number of bits you calculated above, what is the expected size of the uncompressed index? How did you arrive a that number?**  
You can use the previous cell to perform additional calculations

> *Your answer here*

### Compressed Index (2%)

Let us look at the size of the compressed index (`.index`) file 

In [ ]:
print("Size of compressed index", os.path.getsize("output_dir_compressed/BSBI.index"), 'bytes')

**Is the compressed index smaller in size than the one we estimated in the previous section? Why?
How would the size of the compressed index change if variable byte encoding was used on the docIDs instead of the gaps?**

> *Your answer here*

## Improving performance (9%)

In this section you'll write a few sentences (about 5) to answer each of the following questions. Each question is worth 3% of the assignment. 

### Q1
**In this PA we asked you to use each sub-directory as a block and build an index for one block at a time. Can you discuss the trade-off of different sizes of blocks? Is there a general strategy when we are working with limited memory but want to minimize indexing time?**

> Your answer here

### Q2
**Is there a part of your indexing program that limits its scalability to larger datasets?**

> Your answer here

### Q3
**Describe other parts of the indexing process that you can optimize for indexing time/scalability and retrieval time.**

> Your answer here

## Report submission 
You are now ready to submit your report as well. Refer to [submission instructions section](#Submission-instructions). 

# Additional compression methods (10% - Extra credit)

Implement one more index compression method by filling in the `encode` and `decode` methods in `ECCompressedPostings`. 

Here are a few directions  
1. [Section 5.4](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf) in the textbook points to a few techniques (e.g., **gamma-encoding**) and related publications. 
2. You can also look into **Delta Encoding**, a compression technique very similar to Gamma Encoding, which can achieve an even better compression rate. 
3. Also, [ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) is an interesting encoding technique that you could also consider implementing.

You should store each postings list is stored in a multiple of bytes (rather than bits). The positions and lengths in index file are in multiples of bytes. 

In [ ]:
%%tee submission/ec_compressed_postings.py
class ECCompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code

        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code

        ### End your code

**Write a description of your index compression method as well as a discussion of the space/speed trade-off of this additional compression method, in comparison to what we implemented in previous tasks.**

> Your answer here

You will be awarded 8% for answering 20 queries correctly (0.4% each). 2% will be given for the space/speed discussion in your report. 

Note that since the objective of this task is to achieve better compression rate, you'll receive 0% out of 10% if the size of your compressed index is larger than the one using `CompressedPostings`. We will compare against our implementation of `CompressedPostings`, but the results of the autograder tests will be kept visible so you'll know whether you achieve a better compression (and hence the extra credit) before the submission deadline. 

You will be penalized if the retrieval times are abnormally long. We will inspect your code for implementation correctness if necessary.

## EC submission

You are now ready to submit extra-credit as well. You will have to resubmit all code and complete PDF. Refer to [submission instructions section](#Submission-instructions). 